# Kickstarter 

### Loading relevent libraries and dataset

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import date
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from collections import Counter
from nltk import word_tokenize

In [2]:
train = pd.read_csv('train.csv')
train.head(2)

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,backers_count,final_status
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them...,20.0,drawing-for-dollars,False,US,USD,1241333999,1241334017,1240600507,1240602723,3,1
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in ...,"I, Dereck Blackburn will be taking upon an inc...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-re...,False,US,USD,1242429000,1242432018,1240960224,1240975592,2,0


In [3]:
test = pd.read_csv('test.csv')
test.head(2)

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at
0,kkst917493670,Bràthair.,"My first film, of many to come. Trying to purs...",7000.0,brathair,False,US,USD,1449619185,1449619185,1446002581,1446159585
1,kkst1664901914,THE SCREENWRITER,A young man that has earned his master's in sc...,35000.0,the-screenwriter,False,US,USD,1453435620,1453435620,1450297323,1450411620


### Analyzing datasets

In [4]:
print('train length:{0}'.format(len(train)))
print('test length:{0}'.format(len(test)))

train length:108129
test length:63465


In [5]:
print('train length:{0}\n'.format(train.dtypes))
print('test length:{0}'.format(test.dtypes))

train length:project_id                object
name                      object
desc                      object
goal                     float64
keywords                  object
disable_communication       bool
country                   object
currency                  object
deadline                   int64
state_changed_at           int64
created_at                 int64
launched_at                int64
backers_count              int64
final_status               int64
dtype: object

test length:project_id                object
name                      object
desc                      object
goal                     float64
keywords                  object
disable_communication       bool
country                   object
currency                  object
deadline                   int64
state_changed_at           int64
created_at                 int64
launched_at                int64
dtype: object


In [6]:
X = train.copy()
y = test.copy()

In [7]:
X = pd.concat([X,pd.DataFrame(columns=['dl_y','dl_m','dl_d','sc_y','sc_m','sc_d','ca_y','ca_m','ca_d','la_y','la_m','la_d'])])
X.columns

Index(['backers_count', 'ca_d', 'ca_m', 'ca_y', 'country', 'created_at',
       'currency', 'deadline', 'desc', 'disable_communication', 'dl_d', 'dl_m',
       'dl_y', 'final_status', 'goal', 'keywords', 'la_d', 'la_m', 'la_y',
       'launched_at', 'name', 'project_id', 'sc_d', 'sc_m', 'sc_y',
       'state_changed_at'],
      dtype='object')

In [8]:
y = pd.concat([y,pd.DataFrame(columns=['dl_y','dl_m','dl_d','sc_y','sc_m','sc_d','ca_y','ca_m','ca_d','la_y','la_m','la_d'])])
y.columns

Index(['ca_d', 'ca_m', 'ca_y', 'country', 'created_at', 'currency', 'deadline',
       'desc', 'disable_communication', 'dl_d', 'dl_m', 'dl_y', 'goal',
       'keywords', 'la_d', 'la_m', 'la_y', 'launched_at', 'name', 'project_id',
       'sc_d', 'sc_m', 'sc_y', 'state_changed_at'],
      dtype='object')

In [9]:
print(len(y.columns.tolist()))
print(len(X.columns.tolist()))

24
26


In [10]:
X.drop('backers_count', axis=1, inplace=True)

In [11]:
X['dl_y'] = 0
X['dl_m'] = 0
X['dl_d'] = 0
X['sc_y'] = 0
X['sc_m'] = 0
X['sc_d'] = 0
X['ca_y'] = 0
X['ca_m'] = 0
X['ca_d'] = 0
X['la_y'] = 0
X['la_m'] = 0
X['la_d'] = 0

In [12]:
y['dl_y'] = 0
y['dl_m'] = 0
y['dl_d'] = 0
y['sc_y'] = 0
y['sc_m'] = 0
y['sc_d'] = 0
y['ca_y'] = 0
y['ca_m'] = 0
y['ca_d'] = 0
y['la_y'] = 0
y['la_m'] = 0
y['la_d'] = 0

In [13]:
for i, row in X.iterrows():
    date_d = datetime.fromtimestamp(row['deadline']).strftime('%Y-%m-%d')
    date_s = datetime.fromtimestamp(row['state_changed_at']).strftime('%Y-%m-%d')
    date_c = datetime.fromtimestamp(row['created_at']).strftime('%Y-%m-%d')
    date_l = datetime.fromtimestamp(row['launched_at']).strftime('%Y-%m-%d')
    X.at[i,'dl_y'] = date_d[:4]
    X.at[i,'dl_m'] = date_d[5:7]
    X.at[i,'dl_d'] = date_d[8:]
    X.at[i,'sc_y'] = date_s[:4]
    X.at[i,'sc_m'] = date_s[5:7]
    X.at[i,'sc_d'] = date_s[8:]
    X.at[i,'ca_y'] = date_c[:4]
    X.at[i,'ca_m'] = date_c[5:7]
    X.at[i,'ca_d'] = date_c[8:]
    X.at[i,'la_y'] = date_l[:4]
    X.at[i,'la_m'] = date_l[5:7]
    X.at[i,'la_d'] = date_l[8:]

In [14]:
for i, row in y.iterrows():
    date_d = datetime.fromtimestamp(row['deadline']).strftime('%Y-%m-%d')
    date_s = datetime.fromtimestamp(row['state_changed_at']).strftime('%Y-%m-%d')
    date_c = datetime.fromtimestamp(row['created_at']).strftime('%Y-%m-%d')
    date_l = datetime.fromtimestamp(row['launched_at']).strftime('%Y-%m-%d')
    y.at[i,'dl_y'] = date_d[:4]
    y.at[i,'dl_m'] = date_d[5:7]
    y.at[i,'dl_d'] = date_d[8:]
    y.at[i,'sc_y'] = date_s[:4]
    y.at[i,'sc_m'] = date_s[5:7]
    y.at[i,'sc_d'] = date_s[8:]
    y.at[i,'ca_y'] = date_c[:4]
    y.at[i,'ca_m'] = date_c[5:7]
    y.at[i,'ca_d'] = date_c[8:]
    y.at[i,'la_y'] = date_l[:4]
    y.at[i,'la_m'] = date_l[5:7]
    y.at[i,'la_d'] = date_l[8:]

In [15]:
X.drop(['deadline','state_changed_at','created_at','launched_at'], axis=1, inplace=True)
X.columns

Index(['ca_d', 'ca_m', 'ca_y', 'country', 'currency', 'desc',
       'disable_communication', 'dl_d', 'dl_m', 'dl_y', 'final_status', 'goal',
       'keywords', 'la_d', 'la_m', 'la_y', 'name', 'project_id', 'sc_d',
       'sc_m', 'sc_y'],
      dtype='object')

In [16]:
y.drop(['deadline','state_changed_at','created_at','launched_at'], axis=1, inplace=True)
y.columns

Index(['ca_d', 'ca_m', 'ca_y', 'country', 'currency', 'desc',
       'disable_communication', 'dl_d', 'dl_m', 'dl_y', 'goal', 'keywords',
       'la_d', 'la_m', 'la_y', 'name', 'project_id', 'sc_d', 'sc_m', 'sc_y'],
      dtype='object')

In [17]:
X['duration'] = 0
for i, row in X.iterrows():
    dur = date(X.iloc[i]['dl_y'],X.iloc[i]['dl_m'],X.iloc[i]['dl_d']) - date(X.iloc[i]['ca_y'],X.iloc[i]['ca_m'],X.iloc[i]['ca_d'])
    X.at[i,'duration'] = str(dur).split(' ')[0]

In [18]:
y['duration'] = 0
for i, row in y.iterrows():
    dur = date(y.iloc[i]['dl_y'],y.iloc[i]['dl_m'],y.iloc[i]['dl_d']) - date(y.iloc[i]['ca_y'],y.iloc[i]['ca_m'],y.iloc[i]['ca_d'])
    y.at[i,'duration'] = str(dur).split(' ')[0]

In [19]:
X.drop(['dl_d','sc_d','ca_d','la_d'], axis=1, inplace=True)

In [20]:
y.drop(['dl_d','sc_d','ca_d','la_d'], axis=1, inplace=True)

In [21]:
X['disable_communication'].replace([False,True],[0,1], inplace=True)

In [22]:
y['disable_communication'].replace([False,True],[0,1], inplace=True)

In [23]:
X.dropna(axis=0, how='any',inplace=True)
X.isnull().sum()

ca_m                     0
ca_y                     0
country                  0
currency                 0
desc                     0
disable_communication    0
dl_m                     0
dl_y                     0
final_status             0
goal                     0
keywords                 0
la_m                     0
la_y                     0
name                     0
project_id               0
sc_m                     0
sc_y                     0
duration                 0
dtype: int64

In [24]:
xl = X['country'].unique().tolist()
yl = y['country'].unique().tolist()
cmn = list()
for val in yl:
    if val not in xl:
        cmn.append(val)

In [25]:
for i, row in X.iterrows():
    if row['country'] in cmn:
        X.at[i,'country'] = 'ZZ'

In [26]:
for i, row in y.iterrows():
    if row['country'] in cmn:
        y.at[i,'country'] = 'ZZ'

In [27]:
country_x = pd.get_dummies(X['country'],prefix='cntry')
X = pd.concat([X, country_x], axis=1)

In [28]:
country_y = pd.get_dummies(y['country'],prefix='cntry')
y = pd.concat([y, country_y], axis=1)

In [29]:
X.drop('cntry_DE', axis=1, inplace=True)

In [30]:
y.drop(['cntry_ZZ','cntry_DE'], axis=1, inplace=True)

In [31]:
X.drop('country', axis=1, inplace=True)

In [32]:
y.drop('country', axis=1, inplace=True)

In [33]:
xc = X['currency'].unique().tolist()
yc = y['currency'].unique().tolist()
cmn = list()
for val in yc:
    if val not in xc:
        cmn.append(val)

In [34]:
for i, row in y.iterrows():
    if row['currency'] in cmn:
        y.at[i,'currency'] = 'ZZZ'

In [35]:
currency = pd.get_dummies(X['currency'],prefix='curr')
X = pd.concat([X, currency], axis=1)

In [36]:
currency = pd.get_dummies(y['currency'],prefix='curr')
y = pd.concat([y, currency], axis=1)

In [37]:
y.columns

Index(['ca_m', 'ca_y', 'currency', 'desc', 'disable_communication', 'dl_m',
       'dl_y', 'goal', 'keywords', 'la_m', 'la_y', 'name', 'project_id',
       'sc_m', 'sc_y', 'duration', 'cntry_AU', 'cntry_CA', 'cntry_DK',
       'cntry_GB', 'cntry_IE', 'cntry_NL', 'cntry_NO', 'cntry_NZ', 'cntry_SE',
       'cntry_US', 'curr_AUD', 'curr_CAD', 'curr_DKK', 'curr_EUR', 'curr_GBP',
       'curr_NOK', 'curr_NZD', 'curr_SEK', 'curr_USD', 'curr_ZZZ'],
      dtype='object')

In [38]:
y.drop('curr_ZZZ', axis=1, inplace=True)

In [39]:
X.drop('currency', axis=1, inplace=True)

In [40]:
y.drop('currency', axis=1, inplace=True)

In [41]:
y.fillna(value='',axis=0, inplace=True)

In [42]:
tokens_x = list()
for i, row in X.iterrows():
    toks = row['keywords'].split('-')
    tokens_x.append(toks)

In [43]:
tokens_y = list()
for i, row in y.iterrows():
    toks = row['keywords'].split('-')
    tokens_y.append(toks)

In [44]:
stop_clean_x = list()
for j in tokens_x:
    temp = list()
    for i in j :
        if i.lower() not in stopwords.words('english'):
            temp.append(i.lower())
    stop_clean_x.append(temp)

In [45]:
stop_clean_y = list()
for j in tokens_y:
    temp = list()
    for i in j :
        if i.lower() not in stopwords.words('english'):
            temp.append(i.lower())
    stop_clean_y.append(temp)

In [46]:
lemm_toks_x = list()
lemm = WordNetLemmatizer()
for i in stop_clean_x:
    temp = list()
    for j in i:
        word = lemm.lemmatize(j,'a')
        word = lemm.lemmatize(word,'s')
        word = lemm.lemmatize(word,'r')
        word = lemm.lemmatize(word,'n')
        word = lemm.lemmatize(word,'v')
        temp.append(word)
    lemm_toks_x.append(temp)

In [47]:
lemm_toks_y = list()
lemm = WordNetLemmatizer()
for i in stop_clean_y:
    temp = list()
    for j in i:
        word = lemm.lemmatize(j,'a')
        word = lemm.lemmatize(word,'s')
        word = lemm.lemmatize(word,'r')
        word = lemm.lemmatize(word,'n')
        word = lemm.lemmatize(word,'v')
        temp.append(word)
    lemm_toks_y.append(temp)

In [48]:
master_list_x = list()
for i in lemm_toks_x:
    for j in i:
        master_list_x.append(j)

In [49]:
master_list_y = list()
for i in lemm_toks_y:
    for j in i:
        master_list_y.append(j)

In [50]:
master_filter_x = [i for i in master_list_x if len(i)>2]

In [51]:
master_filter_y = [i for i in master_list_y if len(i)>2]

In [52]:
ck = pd.read_csv('common_keys.csv')
ck_cols = ck.columns.tolist()

In [53]:
count = list()
for val in ck_cols:
    if val not in master_filter_y:
        count.append(val)
count 

['do', 'go', 'la', 'le', 'io', 'us']

In [54]:
for val in count:
    ck_cols.remove(val)

In [55]:
X_f_c = pd.DataFrame(0, index=np.arange(len(X)), columns=ck_cols)

In [56]:
y_f_c = pd.DataFrame(0, index=np.arange(len(y)), columns=ck_cols)

In [57]:
lemm_toks_filter_x = list()
for val in lemm_toks_x:
    temp = list()
    for i in val:
        if len(i) > 2:
            temp.append(i)
    lemm_toks_filter_x.append(temp)

In [58]:
lemm_toks_filter_y = list()
for val in lemm_toks_y:
    temp = list()
    for i in val:
        if len(i) > 2:
            temp.append(i)
    lemm_toks_filter_y.append(temp)

In [59]:
for i, val in enumerate(lemm_toks_x):
    for j in val:
        if j in ck_cols:
            X_f_c.at[i,j] = 1      

In [60]:
for i, val in enumerate(lemm_toks_y):
    for j in val:
        if j in ck_cols:
            y_f_c.at[i,j] = 1      

In [61]:
X.reset_index(drop=True,inplace=True)

In [62]:
X = pd.concat([X, X_f_c], axis=1)

In [63]:
y = pd.concat([y.reset_index(drop=True), y_f_c.reset_index(drop=True)], axis=1)

In [64]:
X.drop(['project_id','name','desc','keywords'], axis=1, inplace=True)

In [65]:
y.drop(['project_id','name','desc','keywords'], axis=1, inplace=True)

In [66]:
Xli = X.columns.tolist()
yli = y.columns.tolist()

In [67]:
X.to_csv('train_prep.csv', index=False)

In [68]:
y.to_csv('test_prep.csv', index=False)